# Tests for tracer 

In [1]:
import random
from pto.core.base import tracer, Dist

In [2]:
# Test tracing

# import Dist

''' This shows that successive calls to sample of random primitives are stored sequentially in the trace. '''

def random_program():
    print('CALL 1: ', tracer.sample('pos 1', Dist(random.random)))
    print('CALL 2: ', tracer.sample('pos 2', Dist(random.choice, ['a','b','c'])))
    print('CALL 3: ', tracer.sample('pos 3', Dist(random.randint, 1, 10)))
    
# create trace
''' Running the random program with an empty trace creates a new trace '''
print("CREATE TRACE")
trace = {}
tracer.play(random_program, trace)
print(tracer.TRACE,'\n')

# re-play trace
''' Rewinding the trace and rerunning the random program use the trace to produce the same program output '''
print("RE-PLAY TRACE")
tracer.play(random_program, trace)
print()

# change and fix trace
''' Altering the trace with an annotation inconsistent with the random calls and replaying it fix the trace '''
print("CHANGE AND FIX TRACE")
trace['pos 1'] = Dist(random.randint, 1, 3, val = 2)
print(trace,'\n')

tracer.play(random_program, trace)
print(trace,'\n')

CREATE TRACE
CALL 1:  0.7396417511569042
CALL 2:  b
CALL 3:  7
{'pos 1': Dist(random, (),  val=0.7396417511569042), 'pos 2': Dist(choice, (['a', 'b', 'c'],),  val=b), 'pos 3': Dist(randint, (1, 10),  val=7)} 

RE-PLAY TRACE
CALL 1:  0.7396417511569042
CALL 2:  b
CALL 3:  7

CHANGE AND FIX TRACE
{'pos 1': Dist(randint, (1, 3),  val=2), 'pos 2': Dist(choice, (['a', 'b', 'c'],),  val=b), 'pos 3': Dist(randint, (1, 10),  val=7)} 

repair:  ('randint', (1, 3)) ('random', ())
CALL 1:  0.7760260217088009
CALL 2:  b
CALL 3:  7
{'pos 1': Dist(random, (),  val=0.7760260217088009), 'pos 2': Dist(choice, (['a', 'b', 'c'],),  val=b), 'pos 3': Dist(randint, (1, 10),  val=7)} 



In [3]:
''' This shows that PTO raises an exception when the generator tries to use the same name for more than one trace entry, which is not allowed. '''

def random_program():
    print('CALL 1: ', tracer.sample('pos 1', Dist(random.random)))
    print('CALL 2: ', tracer.sample('pos 1', Dist(random.choice, ['a','b','c'])))
    print('CALL 3: ', tracer.sample('pos 3', Dist(random.randint, 1, 10)))

# run program without tracing (no check on names)    
print(tracer.TRACE,'\n')    
random_program()

# run program with tracing (check on names)    
print(tracer.TRACE,'\n')
tracer.play(random_program, tracer.TRACE)

{'pos 1': Dist(random, (),  val=0.7760260217088009), 'pos 2': Dist(choice, (['a', 'b', 'c'],),  val=b), 'pos 3': Dist(randint, (1, 10),  val=7)} 

CALL 1:  0.6342729432694968
CALL 2:  c
CALL 3:  8
{'pos 1': Dist(random, (),  val=0.7760260217088009), 'pos 2': Dist(choice, (['a', 'b', 'c'],),  val=b), 'pos 3': Dist(randint, (1, 10),  val=7)} 

CALL 1:  0.7760260217088009


AssertionError: PTO ERROR: NAME 'pos 1' IN TRACE NOT UNIQUE

In [4]:
# fix tracer after the exception
tracer.TRACER_ACTIVE = False